In [1]:
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.windows import from_bounds
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import numpy as np
from patchify import patchify

import torch
from torch.functional import F
from torch import nn
from torch.utils.data import DataLoader,TensorDataset
import fiona
from shapely.geometry import shape
import cv2
from pytorch_segmentation.inference import mosaic_to_raster
from pytorch_segmentation.data.inference_dataset import SatInferenceDataset
from pytorch_segmentation.utils.helper import shapes_intersecting_with_raster, raster_bounds_to_shape
#from pytorch_segmentation.utils.preprocessing import unpatchify,pad_image_topleft


seed = 42

device = torch.device('cpu')
#device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')


patch_size = [256,256,3] # [x,y,bands]
overlap = 128
padding = 64

bval = (255,255,255)
nworkers = 4
bs = 16


save_dir = "saved_models"


model_name = "unet_23_06_2022_155611"
data_parallel = True
model_path = save_dir + "/" + model_name +  ".pth" #unet_15_03_2022_071331.pth" #unet_24_03_2022_064749.pth
out_path = "data/out/"+model_name

# SA

In [2]:
dataset_path = "data/datasets/inference_data.pkl"
data_path = "/home/jovyan/work/satellite_data/tmp/2018.vrt"

shape_path = "data/label_masks/dataset/test_sa.shp"
#shape_path = "/home/jovyan/work/satellite_data/tmp/shapes/26/2018.shp"
#mask_path = "data/datasets/V3/test/SA/"
mask_path = "/home/jovyan/work/satellite_data/tmp/25/2018_cog.tif"


In [3]:
raster_bounds_to_shape(mask_path,shape_path)

/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# 1. Data Loading and Preprocessing

In [4]:
shape_idxs = shapes_intersecting_with_raster(shape_path,data_path)
#shape_idxs = None

In [9]:

dataset = SatInferenceDataset(data_file_path=data_path,shape_file=shape_path,shape_idx=shape_idxs,overlap=128,padding=64,nodata=0,pad_mode="reflect")
shapes = dataset.shapes.copy()
dataset.save(dataset_path)

#del dataset


# 2. UNet 


# 2.2 Unet

In [6]:
model_path = "saved_models/" + model_name +  ".pth"
state_dict = torch.load(model_path,map_location="cpu")

if data_parallel:
    # create new OrderedDict that does not contain `module.`
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    model_path = save_dir + "/" + model_name +  "_new.pth"
    torch.save(new_state_dict,model_path)

In [7]:
from pytorch_segmentation.models import UNet
net = UNet(n_channels=patch_size[2], n_classes=2, bilinear=False)

if data_parallel:
    net.load_state_dict(new_state_dict)
else:
    net.load_state_dict(state_dict)

net.eval();



# Multi Process Queue In-Memory

In [8]:
from pytorch_segmentation.inference import mosaic_to_raster
mosaic_to_raster(dataset_path,shapes,net,out_path,device_ids=[0,1],
                bs=150,pin_memory=True,num_workers=5)

Total number of shapes:  5
Shape:  0
Queue PID:  17623


  0%|          | 0/112 [00:00<?, ?it/s]

Start GPU: 0
Start GPU: 1


100%|██████████| 112/112 [00:06<00:00, 17.30it/s]


DONE  0
DONE  1
INFO: Written data/out/unet_23_06_2022_155611/test_sa_1.tif in 0.033 seconds
Shape:  1
Queue PID:  17623


...

  0%|          | 0/616 [00:00<?, ?it/s]

Start GPU: 0
Start GPU: 1


100%|██████████| 616/616 [00:09<00:00, 62.65it/s]

DONE  0
DONE  1


INFO: Written data/out/unet_23_06_2022_155611/test_sa_5.tif in 0.137 seconds
Created Tif file in 1.5200469493865967 seconds: data/out/unet_23_06_2022_155611/mask_24_06_2022_060153.tif


# Rwanda

In [3]:
mask_path = "data/datasets/V3/test/rwanda/"
shape_path = "data/label_masks/dataset/test_rwanda.shp"
data_path = "data/datasets/V3/data_pool/rwanda.vrt"

In [4]:
raster_bounds_to_shape(mask_path,shape_path)

/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# 1. Data Loading and Preprocessing

In [5]:
shape_idxs = shapes_intersecting_with_raster(shape_path,data_path)
#shape_idxs = None

In [6]:

dataset = SatInferenceDataset(data_file_path=data_path,shape_file=shape_path,shape_idx=shape_idxs,overlap=128,padding=64,nodata=0,pad_mode="reflect")
shapes = dataset.shapes.copy()
dataset.save(dataset_path)

#del dataset



# 2. UNet 


# 2.2 Unet

In [16]:
model_path = "saved_models/" + model_name +  ".pth"
state_dict = torch.load(model_path,map_location="cpu")

if data_parallel:
    # create new OrderedDict that does not contain `module.`
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    model_path = save_dir + "/" + model_name +  "_new.pth"
    torch.save(new_state_dict,model_path)

In [17]:
from pytorch_segmentation.models import UNet
net = UNet(n_channels=patch_size[2], n_classes=2, bilinear=False)

if data_parallel:
    net.load_state_dict(new_state_dict)
else:
    net.load_state_dict(state_dict)

net.eval();



# Multi Process Queue In-Memory

In [18]:
from pytorch_segmentation.inference import mosaic_to_raster
mosaic_to_raster(dataset_path,shapes,net,out_path,device_ids=[0,1],
                bs=150,pin_memory=True,num_workers=5)

Total number of shapes:  28
Shape:  0
Queue PID:  17623


  0%|          | 0/260 [00:00<?, ?it/s]

Start GPU: 0
Start GPU: 1


100%|██████████| 260/260 [00:08<00:00, 31.68it/s]


DONE  0
DONE  1
INFO: Written data/out/unet_23_06_2022_155611/test_rwanda_1.tif in 0.047 seconds
Shape:  1
Queue PID:  17623


...

DONE  0
DONE  1
INFO: Written data/out/unet_23_06_2022_155611/test_rwanda_27.tif in 0.057 seconds
Shape:  27
Queue PID:  17623


  0%|          | 0/70 [00:00<?, ?it/s]

Start GPU: 0
Start GPU: 1


100%|██████████| 70/70 [00:05<00:00, 11.71it/s]


DONE  0
DONE  1
INFO: Written data/out/unet_23_06_2022_155611/test_rwanda_28.tif in 0.024 seconds
Created Tif file in 14.844889163970947 seconds: data/out/unet_23_06_2022_155611/mask_24_06_2022_061357.tif
